<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/%EB%B2%A1%ED%84%B0%EC%9D%98_%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   두 벡터 A, B에 대한 코사인 유사도 = dot(A,B)/(norm(A)*norm(B))



In [21]:
docs=['저는 사과 좋아요',
      '저는 바나나 좋아요',
      '저는 바나나 좋아요 저는 바나나 좋아요']

In [22]:
!pip install konlpy

In [23]:
from konlpy.tag import Okt
okt=Okt()

vocab=list(set(word for doc in docs for word in doc.split()))
vocab.sort()
#단어사전 만들기

In [24]:
import pandas as pd
from math import log

n=len(vocab)

def tf(t, d):
  return d.count(t)

def idf(t):
  df=0
  for doc in docs:
    if t in doc:
      df+=1
    #df+= t in doc
  return log(n/(1+df))

def tfidf(t, d):
  return tf(t,d)*idf(t)
#tf, idf, tf-idf 만들기

In [25]:
result=[]
for doc in docs:
  result.append([])
  d=doc
  for word in vocab:
    t=word
    result[-1].append(tf(t, d))
tf_=pd.DataFrame(result, columns=vocab)
tf_
#tf 데이터 프레임 만들기

,바나나,사과,저는,좋아요
0,0,1,1,1
1,1,0,1,1
2,2,0,2,2


In [26]:
result=[]
for doc in docs:
  d=doc
  result.append([])
  for word in vocab:
    t=word
    result[-1].append(tfidf(t, d))
tfidf_=pd.DataFrame(result, columns=vocab)
tfidf_
#tfidf 데이터프레임 만들기

,바나나,사과,저는,좋아요
0,0.000000,0.693147,0.0,0.0
1,0.287682,0.000000,0.0,0.0
2,0.575364,0.000000,0.0,0.0


In [28]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A,B):
  return dot(A,B)/(norm(A)*norm(B))

In [34]:
text=np.array(tf_)

In [36]:
print('문서 1과 문서 2의 유사도 : ', cos_sim(text[0], text[1]))
print('문서 1과 문서 3의 유사도 : ', cos_sim(text[0], text[2]))
print('문서 2과 문서 3의 유사도 : ', cos_sim(text[1], text[2]))
#유클리드 유사도는 문서의 길이에 영향을 받지만 코사인 유사도는 벡터의 방향에 초점을 둔다.

문서 1과 문서 2의 유사도 :  0.6666666666666667
문서 1과 문서 3의 유사도 :  0.6666666666666667
문서 2과 문서 3의 유사도 :  1.0000000000000002
